## SETI end-to-end narrowband drifting tone detection over multiple raw files
This notebook sequences through multiple raw files and looks for drifting narrowband tones.  

This is a general-purpose notebook for real observations such as Voyager or commensal pointings.

For each test file, notebook "01_seti_end_to_end.ipynb" is called which performs the following:
1) Inputs each RAW file 
2) Runs Rawspec to generate a filterbank .h5 spectrogram file with specified fine FFT size and integration factor n_sti
3) Runs TurboSETI and/or seticore and compiles a list of detections and compute time (wall clock)

Note that these sets of files can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

The raw files are copied locally from bulk storage if needed, or rawspec can be run directly from bulk storage without copying locally. 
If local space is limited, raw files can be deleted after each run of this notebook. 

Plots are generated in each "01_seti_end_to_end.ipynb" call.

In [1]:
%load_ext autoreload
%autoreload 2

parameters_are_undefined = False     # use parameter defaults settings in 01_seti_end_to_end.ipynb

delete_raw_file_after_use = True

display_figs01 = False
plot_dets1 = True
plot_dets2 = True
verbose = True
plot_sg_psd_zoom = True
plot_psd_zoom = False
plot_sg_zoom = False
plot_sg_wb = True
plot_psd_wb = True
zoom_bw_MHz = .012  # +/- 6 KHz
max_n_zoom_plots = 50

run_turbo = False
fb_ext = '.h5'

search_max_drift = 10    # Hz/sec
search_min_drift = -10   # ignored except for plotting

delete_raw_file_enable = True   # =True will delete raw file from $RAWDIR, if not present initially in $RAWDIR,
                                # and copied from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False will keep raw file in $RAWDIR regardless
delete_h5_sg_file = True        # =True h5 sg file will be deleted after finishing (reduce file space in $SGDIR)

enable_raw_backup_copy = False  # =True copy to $RAWDIR if needed from $RAW_BACKUP_BASE_DIR/raw_backup_subdir
                                # =False rawspec will read directly from $RAW_BACKUP_BASE_DIR/raw_backup_subdir without file copy

# settings for GBT, override otherwise
fine_fft_size = 1024*1024
n_sti = 4
# n_sti = 32

if (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015'] # 23.7 dB, -.504 Hz/sec
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc3_guppi_57386_VOYAGER1_0004'] 
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (1):
    raw_backup_subdir = 'raw_multichirp/'
    delete_raw_file_enable = False
    raw_file_stem_list = [
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec',
        'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec']
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-183.25sec',
        # 'gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec']
    fig_dir = './plots/'
    n_sti = 16
elif (0):
    raw_backup_subdir = 'raw_voyager/'
    raw_file_stem_list = ['blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011', # 24.4 dB, -.279 Hz/sec
                        'blc23_guppi_59046_80354_DIAG_VOYAGER-1_0012', # no det for 0000
                        'blc23_guppi_59046_80672_DIAG_VOYAGER-1_0013', # 20.93 dB, -.52 Hz/sec
                        'blc23_guppi_59046_80989_DIAG_VOYAGER-1_0014', # no det for 0000
                        'blc23_guppi_59046_81310_DIAG_VOYAGER-1_0015', # 26 dB, -.44 Hz/sec Up to 14 files!
                        'blc23_guppi_59046_81628_DIAG_VOYAGER-1_0016'] # no det for 0000
    fig_dir = './voyager_plots/'
    zoom_bw_MHz = .100   # +/- 50 KHz
elif (1):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0010',
                          'blc2_guppi_57388_HIP113357_0010',
                          'blc3_guppi_57388_HIP113357_0010',
                          'blc4_guppi_57388_HIP113357_0010',
                          'blc5_guppi_57388_HIP113357_0010',
                          'blc6_guppi_57388_HIP113357_0010'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_57388_HIP/'
    raw_file_stem_list = ['blc1_guppi_57388_HIP113357_0012',
                          'blc2_guppi_57388_HIP113357_0012',
                          'blc3_guppi_57388_HIP113357_0012',
                          'blc4_guppi_57388_HIP113357_0012',
                          'blc5_guppi_57388_HIP113357_0012',
                          'blc6_guppi_57388_HIP113357_0012'] 
    fig_dir = './raw_gbt_57388_HIP/'
    fine_fft_size = 1033216     # =1009*1024, believe it or not
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_HIP/'
    raw_file_stem_list = ['blc40_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc41_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc42_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc43_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc44_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc45_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc46_guppi_59103_03394_DIAG_HIP95631_0015',
                          'blc47_guppi_59103_03394_DIAG_HIP95631_0015'] 
    fig_dir = './raw_gbt_59103_HIP/'
elif (0):
    raw_backup_subdir = 'raw_gbt_59103_Kepler/'
    raw_file_stem_list = ['blc40_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc41_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc42_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc44_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc46_guppi_59103_01984_DIAG_KEPLER-160_0010',
                          'blc47_guppi_59103_01984_DIAG_KEPLER-160_0010'] 
    fig_dir = './raw_gbt_59103_Kepler/'
elif (1):
    raw_backup_subdir = 'raw_gbt_59103_TIC/'
    raw_file_stem_list = ['blc40_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc41_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc42_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc43_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc44_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc45_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc46_guppi_59103_03721_DIAG_TIC159107668_0016',
                          'blc47_guppi_59103_03721_DIAG_TIC159107668_0016'] 
    fig_dir = './raw_gbt_59103_TIC/'

n_raw_file = len(raw_file_stem_list)

print('raw_file_stem_list=')
for i_raw in range(n_raw_file):
    print(f'{raw_file_stem_list[i_raw]}')
             
print(f'Figures will appear in in {fig_dir}')

raw_file_stem_list=
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-91.63sec
Figures will appear in in ./plots/


#### Run end-to-end test over parameter sweeps

In [2]:

for i_raw in range(n_raw_file):

    raw_file_stem = raw_file_stem_list[i_raw]
    print(f'\n\nRaw file {i_raw+1} of {n_raw_file}: {raw_file_stem}\n\n')

    try:      
        %run ./01_seti_end_to_end.ipynb
    except:
        print(f'\n\nError Occurred for Raw file {raw_file_stem}, skip\n\n')

    if delete_raw_file_after_use & (not local_raw_file_found):
        # delete raw file to limit local file storage
        try:
            print('Deleting ',raw_file_delete_spec)
            os.system('rm '+raw_file_delete_spec)
        except:
            print(' ')

            



Raw file 1 of 3: gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec


bliss_device_str='"cuda:0"'
Search App=Bliss2, branch='bl0'
search_app_name='bliss_find_hits', test_case='thr10', search_z_threshold=10
sigma_drift = [0.75 1.5  3.  ]
/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec.0000.raw
Raw file found in raw directory
gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-10.0-Hzsec-22.91sec
['gbt', 'chirp80', '0.0040V', '6002.20', '6003.80', 'm10.0', '10.0', 'Hzsec', '22.91sec']
n_chirp = 80 f_start1_MHz = 6002.2 f_stop1_MHz = 6003.8 sig_min_drift = -10.0 sig_max_drift = 10.0
signal_level = 0.004, n_chirp = 80
f_start1_MHz = 6002.2, f_stop1_MHz = 6003.8,  f_start_incr = 20253.164556503296, df_dt_incr = 0.2531645569620249
f_start_truth[0] = 6002200000.0 f_start_truth[-1] = 6003800000.0
drift_rate_truth[0] = -10.0 drift_rate_truth[-1] = 10.0
raw_file_name='/home/kmh4005/data_files/raw_files/gbt-chirp80-0.0040V-6002.20-6003.80-m10.0-

In [3]:
import os

print('All Runs Done')

# Beep 2 times in WSL
os.system("powershell.exe '[console]::beep(261.6,700)'")
os.system("powershell.exe '[console]::beep(261.6,700)'")


All Runs Done


0